In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch
# 下载数据集的路径
path = './DataSet'
# 使用的设备
device = torch.device("cuda")
# 数据预处理,转换为张量并归一化
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 将图像大小调整为 224x224
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 归一化处理

train_set = CIFAR10(root=path, train=True, download=True, transform=transform)
test_set = CIFAR10(root=path, train=False, download=True, transform=transform)
# 加载数据集并设置批次大小
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)
# 修正 print 语句，明确输出内容
print(
    f"训练集的 batch 数量: {len(train_loader)}，每个 batch 中的数据数量: {train_loader.batch_size}")
print(
    f"测试集的 batch 数量: {len(test_loader)}，每个 batch 中的数据数量: {test_loader.batch_size}")

Files already downloaded and verified
Files already downloaded and verified
训练集的 batch 数量: 391，每个 batch 中的数据数量: 128
测试集的 batch 数量: 79，每个 batch 中的数据数量: 128


In [ ]:
import torch
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import Flatten
from torch.nn import Sequential
from torch.nn import BatchNorm2d
from torch.nn import AdaptiveAvgPool2d
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

# 启用异常检测
torch.autograd.set_detect_anomaly(True)


class ResNet(Module):
    class Residual(Module):
        def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
            super().__init__()
            self.features = Sequential(
                Conv2d(input_channels, num_channels, kernel_size=3,padding=1, stride=strides, bias=False),
                BatchNorm2d(num_channels),  # 第一个批量归一化
                ReLU(),
                Conv2d(num_channels, num_channels,kernel_size=3, padding=1, bias=False),
                BatchNorm2d(num_channels),  # 第二个批量归一化
            )
            if use_1x1conv:
                self.conv3 = Conv2d(input_channels, num_channels, kernel_size=1, stride=strides, bias=False)
            else:
                self.conv3 = None

        def forward(self, X):
            Y = self.features(X)
            if self.conv3:
                X = self.conv3(X)
            # 将原地操作改为非原地操作
            Y = Y + X
            return ReLU()(Y)

    def ResNet_Block(self, input_channels, num_channels, num_residuals, first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(self.Residual(input_channels,num_channels, use_1x1conv=True, strides=2))
            else:
                blk.append(self.Residual(num_channels, num_channels))
        return blk

    def __init__(self, num_classes):
        super().__init__()
        self.features = Sequential(
            Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            BatchNorm2d(64),
            ReLU(),
            MaxPool2d(kernel_size=3, stride=2, padding=1),
            *self.ResNet_Block(64, 64, 2, first_block=True),
            *self.ResNet_Block(64, 128, 2),
            *self.ResNet_Block(128, 256, 2),
            *self.ResNet_Block(256, 512, 2)
        )
        # 对特征提取部分的卷积层进行 Kaiming 初始化
        for layer in self.features:
            if isinstance(layer, Conv2d):
                kaiming_uniform_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    torch.nn.init.zeros_(layer.bias)
        self.classifier = Sequential(
            AdaptiveAvgPool2d((1, 1)),
            Flatten(),
            Linear(512, num_classes)
        )
        # 对分类部分的全连接层进行 Xavier 初始化
        for layer in self.classifier:
            if isinstance(layer, Linear):
                xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    torch.nn.init.zeros_(layer.bias)
    def forward(self, X):
        X = self.features(X)
        X = self.classifier(X)
        return X

In [ ]:
# 训练模型
# 定义损失函数和优化器
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import time

def train_model(train_loader, model):
    # 定义损失函数和优化器
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # 训练的轮数
    num_epochs = 10
    # 初始化 running_loss
    running_loss = 0.0
    # 记录训练开始时间
    start_time = time.time()
    # 遍历epoch
    for epoch in range(num_epochs):
        # 遍历训练数据
        for i, (inputs, labels) in enumerate(train_loader):
            # 将数据移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 梯度清零
            optimizer.zero_grad()
            # 计算模型输出
            y_hat = model(inputs)
            # 计算损失
            loss = criterion(y_hat, labels)
            # 反向传播
            loss.backward()
            # 更新参数
            optimizer.step()
            running_loss += loss.item()
        # 打印每个 epoch 的损失
        print(
            f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')
        # 每个 epoch 结束后重置 running_loss
        running_loss = 0.0
    # 记录训练结束时间
    end_time = time.time()
    # 计算训练总时间
    total_time = end_time - start_time
    # 转换为分钟和秒
    minutes = int(total_time // 60)
    seconds = total_time % 60
    print(
        f'Training completed in {minutes} minutes and {seconds:.2f} seconds.')

In [9]:
# 评估模型
from sklearn.metrics import accuracy_score
from numpy import vstack
from numpy import argmax


def evaluate_model(test_loader, model):
    predictions, actuals = list(), list()
    for i, (inputs, labels) in enumerate(test_loader):
        # 将数据移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 计算模型输出
        y_hat = model(inputs)
        # 转换为 numpy 数据类型
        y_hat = y_hat.detach().cpu().numpy()
        actual = labels.cpu().numpy()
        # 转换为类标签
        y_hat = argmax(y_hat, axis=1)
        # 为stack格式化
        actual = actual.reshape((len(actual), 1))
        y_hat = y_hat.reshape((len(y_hat), 1))
        # 存储
        predictions.append(y_hat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # 计算准确率
    acc = accuracy_score(actuals, predictions)
    return acc

In [ ]:
# 产生实例,并且将实例放入GPU
ResNet_model = ResNet(num_classes=10).to(device)
# 训练实例
train_model(train_loader, ResNet_model)
# 评估实例整体准确率
acc = evaluate_model(test_loader, ResNet_model)
print('Overall Accuracy: %.3f' % acc)

Epoch 1/10, Loss: 1.5159172130667644
Epoch 2/10, Loss: 0.8547616166531887
Epoch 3/10, Loss: 0.5726957867669937
Epoch 4/10, Loss: 0.40920635867301763
Epoch 5/10, Loss: 0.28179261267490097
Epoch 6/10, Loss: 0.1763498827414897
Epoch 7/10, Loss: 0.10806727696143453
Epoch 8/10, Loss: 0.06497237058427861
Epoch 9/10, Loss: 0.042235833003192835
Epoch 10/10, Loss: 0.02562270505125146
Training completed in 12 minutes and 38.90 seconds.
Overall Accuracy: 0.841


In [15]:
# 保存训练好的模型
PATH = './DataSet/model/ResNet_model.pth'
torch.save(ResNet_model.state_dict(), PATH)

In [18]:
ResNet_model = ResNet(num_classes=10).to(device)
ResNet_model.load_state_dict(torch.load(PATH, weights_only=True))
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
# 初始化分类准确率统计
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 评估每个类别的准确率
with torch.no_grad():
    ResNet_model.eval()  # 确保模型处于评估模式
    for data in test_loader:
        images, labels = data
        # 将数据移动到GPU
        images, labels = images.to(device), labels.to(device)
        outputs = ResNet_model(images)  # 使用实例化模型
        _, predictions = torch.max(outputs, 1)

        # 将预测结果和标签移动到CPU并转换为numpy数组
        labels = labels.cpu().numpy()
        predictions = predictions.cpu().numpy()

        for label_idx, pred_idx in zip(labels, predictions):
            class_name = classes[label_idx]
            if label_idx == pred_idx:
                correct_pred[class_name] += 1
            total_pred[class_name] += 1

# 打印每个类别的准确率
for classname in classes:
    total = total_pred[classname]
    correct = correct_pred[classname]
    if total == 0:
        print(f"Class {classname} has no test samples")
        continue
    accuracy = 100 * correct / total
    print(f'Accuracy for {classname:5s}: {accuracy:.1f}%')

Accuracy for plane: 87.5%
Accuracy for car  : 93.0%
Accuracy for bird : 78.4%
Accuracy for cat  : 65.7%
Accuracy for deer : 82.1%
Accuracy for dog  : 79.0%
Accuracy for frog : 91.6%
Accuracy for horse: 86.2%
Accuracy for ship : 89.8%
Accuracy for truck: 91.4%
